In [1]:
# Import dependencies
import pandas as pd
import openpyxl
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load data from excel
api_df = pd.read_excel("../data/raw/RiskClassification_Data_Endpoints_V2.xlsx", 
                       "Core_Endpoint",
                       usecols="A:R")
api_df.head()

,api_endpoint_id,api_id,api_vendor_id,api,request_id,method,category,parameters,usage_base,sample_response,tagset,authentication,security_test_category,security_test_result (FALSE=Passed; TRUE=Failed),server_location,hosting_isp,server_name,response_metadata
0,2052,762,276,Natural Language Processing,10789,post,AI & Data Science,"{""Content-Type"": ""application/json"", ""textToDe...",commercial,"{""Successful"":true,""DetectedLanguage_ThreeLett...","DetectedLanguage_FullName,DetectedLanguage_Thr...",header,NaN,NaN,United States,OVH Hosting,Cloudmersive Server,"{""Date"": ""Thu, 13 May 2021 23:23:26 GMT"", ""Pra..."
1,2513,1117,411,Tenor API,7629,get,News & Media,"{""q"": ""Running""}",free,"{\n ""weburl"": ""https://tenor.com/search/runni...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2578,1148,440,ANZ Products,8698,get,Finance & Banking,{},free,"{""data"":{""products"":[{""additionalInformation"":...","links,eligibilityUri,brand,next,overviewUri,da...",header,NaN,NaN,Singapore,Incapsula Inc,istio-envoy,"{""via"": ""kong/0.36-2-enterprise-edition"", ""x-v..."
3,2575,1147,439,NAB Open APIs,8542,get,Finance & Banking,"{""v"": ""1""}",free,<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</...,NaN,header,NaN,NaN,United States,"Akamai Technologies, Inc.",AkamaiGHost,"{""Date"": ""Wed, 18 Mar 2020 07:27:41 GMT"", ""Ser..."
4,2516,1119,413,Translate Text,7733,get,AI & Data Science,"{""lang"": ""en-zh"", ""text"": ""GNE is a good schoo...",free,"{""code"":200,""lang"":""en-zh"",""text"":[""ç½‘å…³ç½‘å...","lang,code,text,text,text,text,text,text",query,NaN,NaN,Russia,Yandex enterprise network,nginx/1.6.2,"{""Date"": ""Mon, 23 Dec 2019 23:10:35 GMT"", ""Ser..."


In [18]:
api_df["api_endpoint_id"].nunique()

113

In [3]:
# Selecting relevant columns
security_test_df = api_df[["api_endpoint_id", "security_test_category", "security_test_result (FALSE=Passed; TRUE=Failed)"]]
security_test_df = security_test_df.rename(columns = {"api_endpoint_id":"endpoint_id",
                                   "security_test_category":"test_category",
                                    "security_test_result (FALSE=Passed; TRUE=Failed)":"test_result"})
security_test_df.head()

,endpoint_id,test_category,test_result
0,2052,NaN,NaN
1,2513,NaN,NaN
2,2578,NaN,NaN
3,2575,NaN,NaN
4,2516,NaN,NaN


In [4]:
# List of unique test categories
list(security_test_df["test_category"].unique())

[nan,
 'Broken Authentication',
 'Buffer Overflow',
 'Insecure Deserialization',
 'SQL Injection',
 'Cross-Site Scripting',
 'XML External Entities']

In [5]:
# Check presence of nan values
security_test_df[security_test_df["test_category"].isnull()]

,endpoint_id,test_category,test_result
0,2052,NaN,NaN
1,2513,NaN,NaN
2,2578,NaN,NaN
3,2575,NaN,NaN
4,2516,NaN,NaN
...,...,...,...
105,2702,NaN,NaN
106,2821,NaN,NaN
107,2743,NaN,NaN
110,2837,NaN,NaN


In [6]:
# Count number of records for each endpoint, including NaN values
security_test_df.groupby('endpoint_id').size()

endpoint_id
2052    1
2410    1
2416    1
2512    1
2513    1
       ..
2830    1
2834    1
2836    1
2837    1
2838    1
Length: 113, dtype: int64

In [7]:
# Cound number of records for each endpoint, excluding NaN values
feat_security_df = security_test_df.groupby('endpoint_id').count()
feat_security_df

,test_category,test_result
endpoint_id,,
2052,0,0
2410,1,1
2416,0,0
2512,0,0
2513,0,0
...,...,...
2830,1,1
2834,0,0
2836,1,1


In [8]:
# Adding rows with no record of security tests
#security_count_df["api_id"] = security_count_df.index
#feat_security_test = pd.merge(feat_security_test, security_count_df, on="api_id", how="outer")

In [9]:
# Adding feature "security_test_conducted": 1 for tests-conducted, 0 for tests-not-conducted
feat_security_df["security_test_conducted"] = 0
feat_security_df.loc[feat_security_df["test_category"] > 0 , "security_test_conducted"] = 1
feat_security_df.drop(columns=["test_category", "test_result"], inplace=True)
feat_security_df.head()

,security_test_conducted
endpoint_id,
2052,0
2410,1
2416,0
2512,0
2513,0


In [10]:
# Adding features for each Security category "test_result":
# 1 for all tests FAIL, 0 for all tests PASS
stacked_df = security_test_df.groupby(["endpoint_id", "test_category"]).mean()
stacked_df

,,test_result
endpoint_id,test_category,
2410,Cross-Site Scripting,0.0
2524,Broken Authentication,0.0
2544,Buffer Overflow,0.0
2583,Insecure Deserialization,0.0
2600,SQL Injection,1.0
2628,Cross-Site Scripting,1.0
2677,Buffer Overflow,0.0
2681,Broken Authentication,0.0
2730,SQL Injection,1.0


In [11]:
stacked_df.unstack()

test_result                                       \
test_category Broken Authentication Buffer Overflow Cross-Site Scripting   
endpoint_id                                                                
2410                            NaN             NaN                  0.0   
2524                            0.0             NaN                  NaN   
2544                            NaN             0.0                  NaN   
2583                            NaN             NaN                  NaN   
2600                            NaN             NaN                  NaN   
2628                            NaN             NaN                  1.0   
2677                            NaN             0.0                  NaN   
2681                            0.0             NaN                  NaN   
2730                            NaN             NaN                  NaN   
2754                            NaN             NaN                  NaN   
2761                            1.0             NaN                  NaN   
2795                            0.0             NaN                  NaN   
2823                            NaN             0.0                  NaN   
2830                            NaN             0.0                  NaN   
2836                            NaN             NaN                  NaN   

                                                                            
test_category Insecure Deserialization SQL Injection XML External Entities  
endpoint_id                                                                 
2410                               NaN           NaN                   NaN  
2524                               NaN           NaN                   NaN  
2544                               NaN           NaN                   NaN  
2583                               0.0           NaN                   NaN  
2600                               NaN           1.0                   NaN  
2628                               NaN           NaN                   NaN  
2677                               NaN           NaN                   NaN  
2681                               NaN           NaN                   NaN  
2730                               NaN           1.0                   NaN  
2754                               0.0           NaN                   NaN  
2761                               NaN           NaN                   NaN  
2795                               NaN           NaN                   NaN  
2823                               NaN           NaN                   NaN  
2830                               NaN           NaN                   NaN  
2836                               NaN           NaN                   0.0

In [12]:
# Combining Security_test_conducted and test_result features
feat_security_df = pd.merge(feat_security_df,
                            stacked_df.unstack(),
                            on="endpoint_id",
                            how="outer")
feat_security_df

,security_test_conducted,"(test_result, Broken Authentication)","(test_result, Buffer Overflow)","(test_result, Cross-Site Scripting)","(test_result, Insecure Deserialization)","(test_result, SQL Injection)","(test_result, XML External Entities)"
endpoint_id,,,,,,,
2052,0,NaN,NaN,NaN,NaN,NaN,NaN
2410,1,NaN,NaN,0.0,NaN,NaN,NaN
2416,0,NaN,NaN,NaN,NaN,NaN,NaN
2512,0,NaN,NaN,NaN,NaN,NaN,NaN
2513,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2830,1,NaN,0.0,NaN,NaN,NaN,NaN
2834,0,NaN,NaN,NaN,NaN,NaN,NaN
2836,1,NaN,NaN,NaN,NaN,NaN,0.0


In [16]:
# Imputing missing values: if a test result is not available, fill '-1' as default
feat_security_df.fillna(-1, inplace=True)
feat_security_df = feat_security_df.round(decimals=1)

In [17]:
# Final Security Test features
feat_security_df

,security_test_conducted,"(test_result, Broken Authentication)","(test_result, Buffer Overflow)","(test_result, Cross-Site Scripting)","(test_result, Insecure Deserialization)","(test_result, SQL Injection)","(test_result, XML External Entities)"
endpoint_id,,,,,,,
2052,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2410,1,-1.0,-1.0,0.0,-1.0,-1.0,-1.0
2416,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2512,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2513,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...
2830,1,-1.0,0.0,-1.0,-1.0,-1.0,-1.0
2834,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2836,1,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
